In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip list | grep torch

pytorch-ignite                     0.5.1
pytorch-lightning                  2.5.0.post0
torch                              2.5.1+cu121
torchaudio                         2.5.1+cu121
torchinfo                          1.8.0
torchmetrics                       1.6.1
torchsummary                       1.5.1
torchtune                          0.5.0
torchvision                        0.20.1+cu121
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
torch.backends.cuda.matmul.allow_tf32 = False

In [4]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [5]:
import wandb
wandb.login(key="c60ac5d8bfe31d20abafbd61d966eca35fe34b65")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sarayu-m (sarayu-m-manipal-institue-of-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
wandb.init(project="gpt2-text-summarization")  # Give your project a name

# ✅ 2️⃣ Define hyperparameters (config)
config = {
    "model_name": "gpt2",
    "learning_rate": 5e-5,
    "batch_size": 8,
    "epochs": 3,
    "max_input_length": 512,
    "max_target_length": 150
}
wandb.config.update(config)  # Log config

wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250328_173924-ord7qkck
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run wandering-bee-17
wandb: ⭐️ View project at https://wandb.ai/sarayu-m-manipal-institue-of-technology/gpt2-text-summarization
wandb: 🚀 View run at https://wandb.ai/sarayu-m-manipal-institue-of-technology/gpt2-text-summarization/runs/ord7qkck


In [7]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [8]:
from datasets import load_dataset  
from transformers import GPT2Tokenizer  
import torch
from torch.utils.data import Dataset, DataLoader

In [9]:
pre_dataset = load_dataset("wikitext","wikitext-103-v1")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [10]:
for i in range(5):
    print(f"Example {i}: {pre_dataset['train'][i]['text']}\n---")

Example 0: 
---
Example 1:  = Valkyria Chronicles III = 

---
Example 2: 
---
Example 3:  Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 

---
Example 4:  The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple a

In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load WikiText dataset
dataset = load_dataset("wikitext", "wikitext-103-v1")  # or "wikitext-2-v1"

# Initialize tokenizer (e.g., GPT-2)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Required for GPT-2
tokenizer.padding_side = "right"

# Filter empty text entries
#dataset = dataset.filter(lambda x: x["text"] is not None and len(x["text"]) > 0)

# Remove empty or whitespace-only entries
dataset = dataset.filter(
    lambda x: x["text"] is not None and len(x["text"].strip()) > 0
)

print("Filtered text:", dataset["train"][1]["text"])
print()
# Should output meaningful text like:
# " = Valkyria Chronicles III =  Senjō no Valkyria 3 : Unrecorded Chronicles ..."

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],  # Tokenize the "text" column
        truncation=True,
        max_length=256,    # Match GPT-2's context window
        padding="max_length",
    )

# Tokenize the dataset
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],  # Only remove the "text" column (others don't exist)
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

Filtered text:  Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 




Map:   0%|          | 0/2891 [00:00<?, ? examples/s]

Map:   0%|          | 0/1165029 [00:00<?, ? examples/s]

Map:   0%|          | 0/2461 [00:00<?, ? examples/s]

In [12]:
print("Filtered text:", tokenized_dataset["train"][1])
print("----------")
sample = tokenized_dataset["train"][1]
input_ids = sample["input_ids"]
attention_mask = sample["attention_mask"]

print("\nTokenized IDs (input_ids):\n", input_ids)
print("Decoded text:\n", tokenizer.decode(input_ids, skip_special_tokens=False))
print("Attention mask:\n", attention_mask)

Filtered text: {'input_ids': [2311, 73, 13090, 645, 569, 18354, 7496, 513, 1058, 1279, 2954, 29, 17740, 357, 4960, 1058, 10545, 230, 99, 161, 254, 112, 5641, 44444, 9202, 25084, 24440, 12675, 11839, 18, 837, 6578, 764, 569, 18354, 7496, 286, 262, 30193, 513, 1267, 837, 8811, 6412, 284, 355, 569, 18354, 7496, 17740, 6711, 2354, 2869, 837, 318, 257, 16106, 2597, 2488, 12, 31, 2712, 2008, 983, 4166, 416, 29490, 290, 6343, 13, 44206, 329, 262, 14047, 44685, 764, 28728, 287, 3269, 2813, 287, 2869, 837, 340, 318, 262, 2368, 983, 287, 262, 569, 18354, 7496, 2168, 764, 12645, 278, 262, 976, 21748, 286, 16106, 290, 1103, 2488, 12, 31, 640, 11327, 355, 663, 27677, 837, 262, 1621, 4539, 10730, 284, 262, 717, 983, 290, 5679, 262, 366, 17871, 5321, 366, 837, 257, 23634, 2422, 4326, 7351, 262, 3277, 286, 7096, 544, 1141, 262, 5498, 1898, 6839, 1810, 508, 1620, 3200, 2042, 4560, 290, 389, 46852, 1028, 262, 11773, 4326, 366, 1279, 2954, 29, 12552, 366, 764, 220, 198, 50256, 50256, 50256, 50256, 50256,

In [13]:
# Print the first tokenized example
print("First tokenized example:\n", tokenized_dataset["train"][0])

# Print a few tokenized input_ids and decoded text
sample = tokenized_dataset["train"][0]
input_ids = sample["input_ids"]
attention_mask = sample["attention_mask"]

print("\nTokenized IDs (input_ids):\n", input_ids)
print("Decoded text:\n", tokenizer.decode(input_ids, skip_special_tokens=False))
print("Attention mask:\n", attention_mask)

First tokenized example:
 {'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [14]:
tokenizer.save_pretrained("tokens")

('tokens/tokenizer_config.json',
 'tokens/special_tokens_map.json',
 'tokens/vocab.json',
 'tokens/merges.txt',
 'tokens/added_tokens.json',
 'tokens/tokenizer.json')

In [15]:
tokenizer = AutoTokenizer.from_pretrained("tokens")


In [16]:
text = "I am Sarayu"
tokens = tokenizer.tokenize(text)
print(tokens)

['I', 'Ġam', 'ĠSar', 'ay', 'u']


In [17]:
vocab_size = tokenizer.vocab_size  # Correct
print(vocab_size)
print("Next code is Transformer's block")

50257
Next code is Transformer's block


In [18]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

# Config (adjust based on your needs)
d_model = 512
n_heads = 4  # Better divisibility with 512
n_layers = 2  # Increased from 3 for better capacity
context_length = 256
dropout = 0.1

print('before the loop!!!')

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        assert d_model % n_heads == 0

        # Combined QKV projection (more efficient)
        self.qkv = nn.Linear(d_model, 3 * d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.tril(torch.ones(context_length, context_length)))

    def forward(self, x):
        B, T, C = x.shape
        qkv = self.qkv(x).split(d_model, dim=2)
        
        # Process Q, K, V
        q, k, v = [y.view(B, T, self.n_heads, self.head_dim).transpose(1, 2) for y in qkv]
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(self.head_dim))
        
        # Apply causal mask
        att = att.masked_fill(self.mask[:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.dropout(att)
        
        y = (att @ v).transpose(1, 2).contiguous().view(B, T, C)
        return self.dropout(self.proj(y))

class GPTBlock(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model),
            nn.Dropout(dropout)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Pre-LN architecture (original GPT-2 style)
        x = x + self.attn(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return x

class GPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.context_length = context_length
        self.wte = nn.Embedding(vocab_size, d_model)
        self.wpe = nn.Embedding(context_length, d_model)  # Learned positional embeddings
        self.blocks = nn.Sequential(*[GPTBlock(d_model, n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)
        
        # GPT-2 style initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        tok_emb = self.wte(idx)
        pos_emb = self.wpe(pos)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)
        
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.context_length:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx



print("Transformer block done")

before the loop!!!
Transformer block done


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
import wandb

In [20]:
print("Changing the train dataset to tensors")
train_dataset = TensorDataset(
    torch.tensor(tokenized_dataset["train"]["input_ids"]),
    torch.tensor(tokenized_dataset["train"]["attention_mask"])
)

Changing the train dataset to tensors


In [21]:
print("Changing the val dataset to tensors")
val_dataset = TensorDataset(
    torch.tensor(tokenized_dataset["validation"]["input_ids"]),
    torch.tensor(tokenized_dataset["validation"]["attention_mask"])
)

Changing the val dataset to tensors


In [22]:
# from torch.utils.data import Subset

# split_ratio = 0.01

# print("Creating smaller validation dataset")
# val_size = int(len(val_dataset) * split_ratio)
# small_val_dataset = Subset(val_dataset, range(val_size))

# print("Creating smaller train dataset")
# train_size = int(len(train_dataset) * split_ratio)
# small_train_dataset = Subset(train_dataset, range(train_size))


In [23]:
# print(f"Small training dataset size: {len(small_train_dataset)}")
# print(f"Small validation dataset size: {len(small_val_dataset)}")

In [24]:
print("Creating dataloaders")
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

Creating dataloaders


In [25]:
# print("Creating dataloaders")
# batch_size = 16  # Adjust based on your GPU memory
# small_train_dataloader = DataLoader(small_train_dataset, batch_size=batch_size, shuffle=True)
# small_val_dataloader = DataLoader(small_val_dataset, batch_size=batch_size)

In [26]:
print("After creating dataloaders")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(tokenizer)
model = GPT(vocab_size=vocab_size).to(device)

After creating dataloaders


In [27]:
print("wandb initialisation")
wandb.init(project="gpt-wikitext", config={
    "vocab_size": vocab_size, 
    "batch_size": batch_size,
})

wandb initialisation


In [28]:
print("Training configuration")
learning_rate = 5e-5
num_epochs = 2
warmup_steps = 1000
total_steps = len(train_dataloader) * num_epochs


Training configuration


In [29]:
print("Optimiser and Scheduler")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

Optimiser and Scheduler


In [30]:
print("Before training")
for epoch in range(num_epochs):
    print("Entered epoch")
    model.train()
    total_loss = 0
    print("Training")
    for batch in train_dataloader:
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        
        # Create targets (shifted input_ids)
        targets = input_ids[:, 1:].contiguous()
        
        optimizer.zero_grad()
        logits, loss = model(input_ids[:, :-1], targets=targets)
        loss = loss * attention_mask[:, :-1].sum(dim=1) / attention_mask[:, :-1].sum()  # Account for padding
        loss = loss.mean()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()

    print("Training done")
    avg_train_loss = total_loss / len(train_dataloader)
    
    # Validation
    print("Entering validation")
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask = batch
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            targets = input_ids[:, 1:].contiguous()
            
            _, loss = model(input_ids[:, :-1], targets=targets)
            loss = loss * attention_mask[:, :-1].sum(dim=1) / attention_mask[:, :-1].sum()
            total_val_loss += loss.mean().item()
    
    print("Validation done")
    avg_val_loss = total_val_loss / len(val_dataloader)
    
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Average training loss: {avg_train_loss:.4f}")
    print(f"Average validation loss: {avg_val_loss:.4f}")

    # Log metrics to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "val_loss": avg_val_loss,
        "learning_rate": scheduler.get_last_lr()[0]
    })

print("Training done")
print("Saving the model")
# Save the model
torch.save(model.state_dict(), "gpt_model.pth")
wandb.save("gpt_model.pth")

print("Finish wandb") # Finish the wandb run
wandb.finish()

Before training
Entered epoch
Training
Training done
Entering validation
Validation done
Epoch 1/2
Average training loss: 0.1118
Average validation loss: 0.0944
Entered epoch
Training
Training done
Entering validation
Validation done
Epoch 2/2
Average training loss: 0.0949
Average validation loss: 0.0901
Training done
Saving the model
Finish wandb


wandb: uploading gpt_model.pth; uploading config.yaml; uploading output.log
wandb: uploading gpt_model.pth
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:         epoch ▁█
wandb: learning_rate █▁
wandb:    train_loss █▁
wandb:      val_loss █▁
wandb: 
wandb: Run summary:
wandb:         epoch 2
wandb: learning_rate 0
wandb:    train_loss 0.09489
wandb:      val_loss 0.0901
wandb: 
wandb: 🚀 View run wandering-bee-17 at: https://wandb.ai/sarayu-m-manipal-institue-of-technology/gpt2-text-summarization/runs/ord7qkck
wandb: ⭐️ View project at: https://wandb.ai/sarayu-m-manipal-institue-of-technology/gpt2-text-summarization
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 1 other file(s)
wandb: Find logs at: ./wandb/run-20250328_173924-ord7qkck/logs
